In [20]:
from pysgx import stocks, info, util
import importlib
import pandas as pd

In [27]:
importlib.reload(info);

In [16]:
df = pd.DataFrame.from_dict(info.get_active(), orient='index').reset_index()
df.columns = ['Ticker', 'MarketCap']

In [21]:
df = util.to_df(info.get_all(), ['Ticker', 'MarketCap'])

In [ ]:
plt = df[df.MarketCap<100000000].MarketCap.plot.hist(bins=50, alpha=0.5, title='SGX Stock Market Cap Distribution - Below(100M SGD)')
plt.set_xlabel("Market Cap (100M)")
plt.get_figure().savefig('mc_100M.png')